In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_trade, cancel_trade, place_accept, close_trading
from store.operations import StoringPool
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))

#close_trading(client, 693, storeAppID, creator)

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 499


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator, token_id=asset_id)
# storeApp.create_app()
# storeAppID = storeApp.app_id

storeAppID = int(os.environ.get("STORE_APP_ID"))
storeApp.app_id = storeAppID
print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Store App ID: 727
Store App Address: 5YFHLDU7ELKNT3N25YKEXEHALZL6C3UEZLKZOSMICARY3ZBXDEPRKAKYWU


In [7]:

appID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction EX55WVKTBBYQUWX7HWJZY3FJFOJGOM6EEVGKB2N7TRBDL34SCNCQ confirmed in round 77844.
App ID: 1473
App Address: DVR5LPRWKLOMAGDSZ6WU5QVX3JGW62VOC2WFDBWOREJ4BXDFMH6DH2SVX4


Alice is setting up and funding token auction...

In [8]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_id=asset_id,
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]
print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction 5SOOSEWVABKXN3N5HWGAHBKNT4ZKWEZN4HMDBHVPIVXSEH7K3VRQ confirmed in round 77846.
Alice's algo balance:  20037596683628  algos


In [9]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100101000, 499: 0}


In [10]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10019403884749  algos
Carla is placing bid for:  1000000  algos


In [11]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

store_app_id 727
new address: 6WUN7376ILJCYKNCWRLDCRIIAECD4SKCBXZWPTBNBWGBTX43EUOMGOKEFY
new private_key: zG7o4ykDfwdqrpiQHx8P0xvcK7GAaap5hyulbY3XdO71qN/v/kLSLCmitFYxRQgBBD5JQg3zZ8wtDYwZ35slHA==
new passphrase: sunset payment vehicle crater wrap limb infant obvious velvet ladder maple rule return enough airport hat crystal bulb noise surge bracket curious inhale abstract hidden
Waiting for confirmation...
Transaction 77L6WWZJQA5RUHLRGTL6B2CFSCB72E5MNQLYXQO55YMJQJZ7I2XA confirmed in round 77849.
Waiting for confirmation...
Transaction EEQ7Y527ASY3LNFZS7Z2M77ZVN6EVI2XUBBKFWEQXLU5DMTFZI6A confirmed in round 77851.
Waiting for confirmation...
Transaction 5CVKCN6MGAEJHE5NA3XXKTQCVHKMVQTEW6BQHN74HGY4DISQS5GA confirmed in round 77853.
obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {'5TG3X5EGVIRUWLL72O4GRLDQKMLXWMKVMCVEFZRTNXQ65MAMW3UNUQ6DUI': 1}, 'Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM': {'67KQREK4UKL3BJFBPTTGGHF5STPU2QQ3KXLF7ZNBIKNHODJQ7K25OV7U5M': 1, 

In [12]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_trade(client, appID, seller, trading_index)
print("Carla cancel trading", trading_index)

Waiting for confirmation...
Transaction BM6T7UNOCHLNQWDZDA5BPP3KZSWXHMYQJLGHTQHRYEASEZLDUL4Q confirmed in round 77857.
Carla cancel trading 6WUN7376ILJCYKNCWRLDCRIIAECD4SKCBXZWPTBNBWGBTX43EUOMGOKEFY


In [13]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=15, price=bidPrice, trading_index=trading_index)

store_app_id 727
new address: 6IWEQJHKDP45O35COSVCBZRQAXKSVCQQGNEUNZJ7MU6RUGUTT4TH4ULBLA
new private_key: XhLs1MvvlLEA7gqxTv8lgfH8C3QhbJ3TzFIvBL2zWE/yLEgk6hv512+idKog5jAF1SqKEDNJRuU/ZT0aGpOfJg==
new passphrase: envelope raccoon tuna witness clap cluster hybrid april stomach wood napkin blossom wheat actual arm history inhale crew practice drill key coconut kiss able advice
Waiting for confirmation...
Transaction BJWMELDFR2XZKRIFK5VDSLJFQEIBZNZWAVF2VEXZUZ2JD27VZEKQ confirmed in round 77859.
Waiting for confirmation...
Transaction WYWG7C45LOU6WHGROYHUDKVYVDCOVRRSFG2YVNIEHNG5IQXPPDVA confirmed in round 77861.
Waiting for confirmation...
Transaction F52T3ZHGZUBDJNH2UQRSBEJXGF6GQHUYA3QC42LG5JHHBVCYPU7Q confirmed in round 77863.
obj {'LOUBIMVHXMWMJNC4ZBKTIQTGNLF36EIXTEVBD7BUHDMIXKSMKQRNIPMQ6I': {'5TG3X5EGVIRUWLL72O4GRLDQKMLXWMKVMCVEFZRTNXQ65MAMW3UNUQ6DUI': 1}, 'Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM': {'67KQREK4UKL3BJFBPTTGGHF5STPU2QQ3KXLF7ZNBIKNHODJQ7K25OV7U5M': 1, 'E4U2

'6IWEQJHKDP45O35COSVCBZRQAXKSVCQQGNEUNZJ7MU6RUGUTT4TH4ULBLA'

In [14]:
place_accept(client, creator, appID, bidder, seller.get_address(), trading_index)

token_amount 15
price 1000000
Waiting for confirmation...
Transaction 723YKJM5LVDPM5R4YLO3AILWCRQXI2PIJM7XMODC24UR4QGN2P2A confirmed in round 77869.
Waiting for confirmation...
Transaction Y63M7AD4SIGCNJXRT4Q5TBPHYLRRUGPBLA36HQACNBUEGHECKAUA confirmed in round 77871.


True

Alice is closing out the auction....

In [15]:
sleep(5)

close_trading(client, appID, creator, [asset_id])

b'assets' [499]
b'accounts' ['4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction G463FMVE2S6F25ACBBZTZY6WJVYTPCZEJSH6N4EJ3TXKVQRPTH5Q confirmed in round 77874.


In [16]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 100097100, 499: 0}


In [17]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 31  for token ID:  499


AssertionError: 

In [ ]:
app_global_state = get_app_global_state(client, appID)
print(app_global_state)
print(encoding.encode_address(app_global_state[b"AS"]))
print(encoding.encode_address(app_global_state[b"S"]))
#print(encoding.encode_address(storeAppState[b"C"]))